In [1]:
import pandas as pd
import numpy as np

import spacy
#reformat to the proper matrix dimension (num. samples, vector dimension of 300)
from scipy.sparse import csr_matrix
from scipy.sparse import hstack

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv("Data/train_processed_tf_idf_bow.csv")
test = pd.read_csv("Data/test_processed_tf_idf_bow.csv")
sub_sample = pd.read_csv("Data/sample_submission.csv")

print (train.shape, test.shape, sub_sample.shape)



(7613, 1842) (3263, 1841) (3263, 2)


In [3]:
train.tail()

,Unnamed: 0,id,keyword,location,text,target,text_clean,hashtags,mentions,links,...,û_,ûª,ûªs,ûªt,ûªve,ûï,ûïwhen,ûïwhen saw,ûò,ûó
7608,7608,10869,None,None,Two giant cranes holding a bridge collapse int...,1,Two giant cranes holding a bridge collapse int...,no,no,http://t.co/STfMbbZFB5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7609,7609,10870,None,None,@aria_ahrary @TheTawniest The out of control w...,1,@aria_ahrary @TheTawniest The out of control w...,no,aria_ahrary TheTawniest,no,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7610,7610,10871,None,None,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,M1.94 [01:04 UTC]?5km S of Volcano Hawaii.,no,no,http://t.co/zDtoyd8EbJ,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7611,7611,10872,None,None,Police investigating after an e-bike collided ...,1,Police investigating after an e-bike collided ...,no,no,no,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7612,7612,10873,None,None,The Latest: More Homes Razed by Northern Calif...,1,The Latest: More Homes Razed by Northern Calif...,no,no,http://t.co/YmY4rSkQ3d,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:

text_data=train['text'].apply(lambda x: str(x))
text_label=train.keyword

In [5]:
# map sentiment to ordinal values
sentiment_map={'positive':1, 'negative':0}
response=text_label.map(sentiment_map)

In [6]:
nlp = spacy.load('en')

In [7]:
#gets the average wordvec
vec1 = [doc.vector for doc in nlp.pipe(text_data, n_threads=50)]
train['word_vec'] = list(np.array(vec1))

In [8]:
X_train=csr_matrix(list(train['word_vec']))
X_train.shape

(7613, 96)

In [9]:
X_train.shape

(7613, 96)

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_train, response, test_size=0.25, random_state=42, stratify=response)

clf= MLPClassifier(alpha=1e-4, hidden_layer_sizes=(6,4), random_state=12,validation_fraction=0)

clf.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
val_pred=clf.predict(X_val.toarray())

cal_error=accuracy_score(y_val,val_pred)
print(cal_error)


submit = sub_sample.copy()
submit.target = y_test
submit.to_csv('submit_glove.csv',index=False)